In [1]:
! python --version

Python 3.8.3


### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
scale = StandardScaler()

### Import Data

In [3]:
# load in data
df = pd.read_csv('https://raw.githubusercontent.com/oliviatrase/Trase_etal_2025_Volatiles/refs/heads/main/20250306_Volatiles_normalized.csv')
df.set_index('Unnamed: 0', drop = True, inplace = True)
df.head()

,SoilType,WCR,Variety,Block,8-Heptadecene,(-)-Menthol,alpha-muurolene,Unknown RI 1061.3,3-Heptyl formate,"1-Hexanol, 2-ethyl-",...,Unknown RI 1125.8,Unknown RI 1045.5,Caryophyllene,Unknown RI 1302.4,p-ethyl acetophenone,Benzothiazole,Unknown RI 899.2,Acetophenone,alpha-Phellandrene,Totals
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A09,CCFallow,no,MastersChoice,2,0.000000,1.198596,4.448340,78.148098,4.227219,23.936098,...,2.393744,11.356703,5.602171,6.200211,6.549249,2.423210,2.025090,2.554586,5.411014,553.749125
A10,CCFallow,yes,MastersChoice,2,0.000000,2.386610,6.055481,60.886522,6.756527,44.031606,...,4.708354,23.892494,38.936720,8.532684,5.054594,9.923057,1.824835,4.371371,6.886684,793.835328
A11,CCFallow,yes,MastersChoice,2,0.000000,2.006937,4.850652,57.391168,13.378667,37.014280,...,1.469423,16.446731,12.913093,8.305178,3.415067,3.346420,1.731841,3.043973,5.432817,626.203326
A12,CCFallow,no,MastersChoice,2,0.000000,0.000000,1.443654,16.542691,0.000000,14.408412,...,1.266995,3.474925,5.016997,2.308813,3.087856,1.191454,1.698009,4.551204,0.775725,205.899641
A13,CCFallow,yes,MastersChoice,2,0.258336,0.000000,3.205233,26.233618,0.000000,27.070512,...,1.635795,4.566233,4.700896,2.643314,2.397482,0.297849,1.825091,4.353896,0.797523,330.690107


### Normalize Data

Experiments were done on different days, and the day to day variation is high. Therefore, in order to create a normalized data set to use with machine learning algorithms, I need to normalize the data by block. To do this, I have chose to drag the within-block average total VOC content toward the whole average total voc content by multiplying each value by a multiplier (here calculated as the whole average total voc value divided by the within-block average total voc value.

In [4]:
## Normalize by Block

## get average total voc content
total_avg = df['Totals'].mean()

## loop through each block
block_df_list = []
for i in range(5):
    ## subset by block
    block = df['Block'].unique()[i]
    block_df = df[df['Block']==block]
    
    ## get avg total VOC content per block
    block_total_avg = block_df['Totals'].mean()
    
    ## divide total voc avg by block avg
    multiplier = total_avg/block_total_avg
    
    ## multiply each value by the multiplier to drag block avg toward total avg
    block_df2 = block_df.drop(['SoilType','Variety','Block','WCR'],axis=1)
    block_df2 = block_df2*multiplier
    
    ## replace WCR column
    block_df2['WCR'] = block_df['WCR']

    ## append new normalized block df to list
    block_df_list.append(block_df2)

## Concatenate list of block data frames into one dataframe
df_norm = pd.concat(block_df_list)

df_norm.head()

,8-Heptadecene,(-)-Menthol,alpha-muurolene,Unknown RI 1061.3,3-Heptyl formate,"1-Hexanol, 2-ethyl-",3-Octanone,Unknown RI 946.6,"Ethanol, 2-(2-ethoxyethoxy)-",1-Decanol,...,Unknown RI 1045.5,Caryophyllene,Unknown RI 1302.4,p-ethyl acetophenone,Benzothiazole,Unknown RI 899.2,Acetophenone,alpha-Phellandrene,Totals,WCR
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A09,0.000000,1.540462,5.717104,100.437660,5.432915,30.763202,0.797387,0.000000,5.756338,13.404085,...,14.595885,7.200034,7.968647,8.417240,3.114363,2.602690,3.283210,6.954355,711.690596,no
A10,0.000000,3.067324,7.782638,78.252702,8.683638,56.590392,3.821772,0.000000,7.079588,10.272609,...,30.707161,50.042332,10.966394,6.496276,12.753332,2.345318,5.618183,8.850918,1020.254682,yes
A11,0.000000,2.579360,6.234165,73.760395,17.194559,47.571569,2.926217,0.000000,5.003152,7.435354,...,21.137702,16.596192,10.673997,4.389120,4.300893,2.225800,3.912181,6.982377,804.810334,yes
A12,0.000000,0.000000,1.855416,21.261031,0.000000,18.518010,0.755647,3.411191,0.482312,5.049466,...,4.466050,6.447956,2.967337,3.968581,1.531284,2.182318,5.849308,0.996979,264.626762,no
A13,0.332019,0.000000,4.119436,33.716025,0.000000,34.791620,2.008455,0.151656,1.544085,4.364320,...,5.868623,6.041695,3.397246,3.081297,0.382802,2.345647,5.595723,1.024994,425.010223,yes


### Prepare Data for ML

In [5]:
## Transform WCR treatments into codes for ML
wcrcodes = {'yes': 1,'no': 0, 0: 10}
df_norm['WCRcode'] = [wcrcodes[item] for item in df_norm['WCR']]

## Drop WCR treatment column and keep WCR code column
df_norm.drop('WCR', axis=1, inplace=True) 

## Drop Totals column
df_norm.drop('Totals', axis = 1, inplace = True)

### Check Correlations of VOCs with presence of WCR

In [6]:
## get correlations
correlations = df_norm.corr()

## Extract correlations with the target variable 'WCRcode'
target_correlations = correlations['WCRcode'].drop('WCRcode') 

## take absolute values of correlations
target_correlations = target_correlations.abs()

## Convert the series to a DataFrame with column names for better visualization
target_correlations_df = target_correlations.to_frame().reset_index()
target_correlations_df.columns = ['Feature', 'Correlation_with_WCRcode']

## Sort by absolute value of correlation
target_correlations_df = target_correlations_df.reindex(target_correlations_df.Correlation_with_WCRcode.abs().sort_values(ascending=False).index)

target_correlations_df.head()

,Feature,Correlation_with_WCRcode
48,"2,5-Cyclohexadiene-1,4-dione, 2,6-bis(1,1-dime...",0.315690
19,beta-Selinene,0.259624
32,Germacrene D,0.259008
23,"(E)-4,8-Dimethylnona-1,3,7-triene",0.258751
1,(-)-Menthol,0.253242


The highest correlations of VOCs with WCR presence are 25%-32%

### Split data into feature table and target labels for ML

In [7]:
## Sample data frame creation
## 'X' contains features (chemical compounds) and 'y' contains target labels
X = df_norm.drop('WCRcode',axis=1)
y = df_norm['WCRcode']

### Split data into training data and testing data

For these data, the the model will be trained on 80% of the data and tested on 20% of the data

In [8]:
## Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Select features which have higher correlation with WCR presence

Here we select features which have a higher correlation with WCR presence (over 25%).

In [9]:
# Set threshold for significant correlation with target
threshold_target_corr = 0.25
selected_features = target_correlations[abs(target_correlations) > threshold_target_corr].index.tolist()
print(f"Selected features correlated with WCR presence: ({selected_features})")

Selected features correlated with WCR presence: (['(-)-Menthol', 'beta-Selinene', '(E)-4,8-Dimethylnona-1,3,7-triene', 'Germacrene D', '2,5-Cyclohexadiene-1,4-dione, 2,6-bis(1,1-dimethylethyl)-'])


### Calculate the variance inflation factor (VIF) to check for multicollinearity

Here we determine that we can keep any features which have a VIF of less than 10.

In [10]:
## Subset normalized data frame by selecting only highly correlated features
selected_data = df_norm[selected_features]

## Initialize the dataframe to store VIF values
vif_data = pd.DataFrame()
vif_data["Feature"] = selected_data.columns

## Calculate VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(selected_data.values, i) for i in range(len(selected_data.columns))]

## Set a threshold for VIF to detect multicollinearity
threshold_vif = 10
selected_features = vif_data[vif_data["VIF"] < threshold_vif]["Feature"].tolist()
print(f"Selected features with VIF below threshold ({selected_features})")

Selected features with VIF below threshold (['(-)-Menthol', 'beta-Selinene', '(E)-4,8-Dimethylnona-1,3,7-triene', 'Germacrene D', '2,5-Cyclohexadiene-1,4-dione, 2,6-bis(1,1-dimethylethyl)-'])


### Subset training data and testing data using the selected features

In [11]:
## Change the train and test sets to only have the selected features
X_train = X_train[selected_features]
X_test = X_test[selected_features]

### Define a scoring function to optimize for accuracy and recall

Given that the purpose of this algorithm is to generate hypotheses determining which volatiles are likely signals of WCR infestation, it is important to include recall as well as accuracy in determining success. 

In [12]:
## Define scoring function
def custom_scoring_function(y_true, y_pred):
    ## Extract values from confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    ## Calculate accuracy
    accuracy = (tp + tn) / (tp+tn+fp+fn)
    
    ## Calculate recall score
    recall = tp / (tp+fn)
    
    ## Calculate score
    score = accuracy+recall
    return score

## Create scorer
scorer = make_scorer(custom_scoring_function)

### Define classifiers and grid parameters for grid search

Here I would like to test a Random Forest algorithm, a Support Vector Machine, and a K-Nearest Neighbors algorithm to determine which algorith provides the best score (Accuracy + Recall)

In [13]:
## Set Random State
random_state = 42

## Define classifiers
classifiers = {
    'RandomForest': RandomForestClassifier(random_state=random_state),
    'SVM': SVC(random_state=random_state),
    'KNN': KNeighborsClassifier()
}

## Create pipeline with scaling and classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', None)
])

## Define parameter grids for grid search
param_grids = {
    'RandomForest': {'classifier__n_estimators': [1,2,3,4,5],
                     'classifier__max_features': ['sqrt', 'log2'],
                     'classifier__max_depth' : [1,2,3,4],
                     'classifier__criterion' :['gini', 'entropy']},
    'SVM': {'classifier__C': [0.1,0.5,1], 
            'classifier__kernel': ['linear', 'rbf']},
    'KNN': {'classifier__n_neighbors': [2,3,4]}
}

### Run Grid Search for each of the classifiers

I use the GridSearchVC function and cross validate 10 times for each combination of parameters.

In [14]:
results = {}
for name, clf in classifiers.items():
    print(f'Currently running {name}')
    pipeline.set_params(classifier=clf)
    param_grid = param_grids[name]
    grid_search = GridSearchCV(pipeline, param_grid, scoring=scorer, cv=10)
    grid_search.fit(X_train, y_train)
    results[name] = grid_search

Currently running RandomForest
Currently running SVM
Currently running KNN


### Get best parameters for Random Forest

In [15]:
criterion = results['RandomForest'].best_params_['classifier__criterion']
max_depth = results['RandomForest'].best_params_['classifier__max_depth']
max_features = results['RandomForest'].best_params_['classifier__max_features']
n_estimators = results['RandomForest'].best_params_['classifier__n_estimators']
print(f'Best parameters for RandomForest: criterion={criterion}, max_depth={max_depth}, max_features={max_features}, n_estimators={n_estimators}')

Best parameters for RandomForest: criterion=entropy, max_depth=1, max_features=sqrt, n_estimators=1


### Run Random Forest Model using Best Parameters

In [25]:
##  Train the model
model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, max_features=max_features, random_state=random_state)
model.fit(X_train, y_train)

## Run the model on the test data
y_pred = model.predict(X_test)

## Look at accuracy/precision/recall/confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f'True Negatives: {tn}, False Positives: {fp}, False Negatives: {fn}, True Positives: {tp}')
print(f'Accuracy: {round((tn+tp)/(tn+fp+fn+tp),2)}')
print(f'Precision: {round(tp/(tp+fp),2)}')
print(f'Recall: {round(tp/(tp+fn),2)}')

print(f'Custom Score: {round(((tn+tp)/(tn+fp+fn+tp))+(tp/(tp+fn)),2)}')

True Negatives: 6, False Positives: 27, False Negatives: 1, True Positives: 44
Accuracy: 0.64
Precision: 0.62
Recall: 0.98
Custom Score: 1.62


### Get best parameters for Support Vector Machine

In [26]:
c_value = results['SVM'].best_params_['classifier__C']
kernel = results['SVM'].best_params_['classifier__kernel']
print(f'Best parameters for SVM: C={c_value}, kernel={kernel}')

Best parameters for SVM: C=0.1, kernel=linear


### Run SVM using Best Parameters

In [27]:
##  Train the model
model_svm = SVC(C=c_value,kernel=kernel)
model_svm.fit(X_train, y_train)

## Run the model on the test data
y_pred_svm = model_svm.predict(X_test)

## Look at accuracy/precision/recall/confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_svm).ravel()
print(f'True Negatives: {tn}, False Positives: {fp}, False Negatives: {fn}, True Positives: {tp}')
print(f'Accuracy: {round((tn+tp)/(tn+fp+fn+tp),2)}')
print(f'Precision: {round(tp/(tp+fp),2)}')
print(f'Recall: {round(tp/(tp+fn),2)}')

print(f'Custom Score: {round(((tn+tp)/(tn+fp+fn+tp))+(tp/(tp+fn)),2)}')

True Negatives: 13, False Positives: 20, False Negatives: 9, True Positives: 36
Accuracy: 0.63
Precision: 0.64
Recall: 0.8
Custom Score: 1.43


### Get best parameters for KNN

In [28]:
n_value = results['KNN'].best_params_['classifier__n_neighbors']
print(f'Best parameters for KNN: n_neighbors={n_value}')

Best parameters for KNN: n_neighbors=3


### Run KNN using Best Parameters

In [29]:
##  Train the model
model_knn = KNeighborsClassifier(n_neighbors=n_value)
model_knn.fit(X_train, y_train)

## Run the model on the test data
y_pred_knn = model_knn.predict(X_test)

## Look at accuracy/precision/recall/confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_knn).ravel()
print(f'True Negatives: {tn}, False Positives: {fp}, False Negatives: {fn}, True Positives: {tp}')
print(f'Accuracy: {round((tn+tp)/(tn+fp+fn+tp),2)}')
print(f'Precision: {round(tp/(tp+fp),2)}')
print(f'Recall: {round(tp/(tp+fn),2)}')

print(f'Custom Score: {round(((tn+tp)/(tn+fp+fn+tp))+(tp/(tp+fn)),2)}')

True Negatives: 20, False Positives: 13, False Negatives: 11, True Positives: 34
Accuracy: 0.69
Precision: 0.72
Recall: 0.76
Custom Score: 1.45


#### Random Forest outperformed SVM and KNN

### Check Feature Importance for Random Forest Classifier

In [30]:
## Get feature importances
importances = model.feature_importances_

## Put importances into data frame
feature_imp_df = pd.DataFrame({'Feature': X_train.columns, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False) 
print(feature_imp_df)

                                             Feature  Gini Importance
0                                        (-)-Menthol              1.0
1                                      beta-Selinene              0.0
2                  (E)-4,8-Dimethylnona-1,3,7-triene              0.0
3                                       Germacrene D              0.0
4  2,5-Cyclohexadiene-1,4-dione, 2,6-bis(1,1-dime...              0.0


Menthol is the only feature important for the model